# Extracción de tópicos y representación vectorial de noticias

In [183]:
import psycopg2
import sys
import pandas as pd
import spacy
import gensim
import numpy as np
from gensim import corpora
from gensim import models
import pickle  # to save the dictionary
import pyLDAvis.gensim

In [184]:
# --------------------------------------------------------------------------- #
# utility functions

# Función para ingesta desde base de datos
def get_news_data(dml_query,limit=100):

    con = psycopg2.connect("host='192.168.1.70' dbname='newspapers_oem1' \
    user='kenny' password='secr3tp455' port = 5433")   

    cur = con.cursor()
    #cur.execute("SELECT * FROM information_schema.sql_packages")
    #Select * from public.news_content1  WHERE newspaper_name = 'Periódico ABC' AND news_date_reported < '2018-01-01'
    #cur.execute("SELECT * FROM public.news_content1 WHERE section_name = 'opinion' AND importance_level = 'Nacional' AND newspaper_name = 'La Jornada' AND news_date_reported > '2018-11-10' LIMIT 160")
    
    # TEMPLATE QUERY !!!
    # cur.execute("SELECT * FROM public.news_content1 WHERE section_name = 'opinion' \
    # AND importance_level = 'Nacional' AND news_date_reported > '2018-11-10' LIMIT 250")
    
    dml_query=dml_query+" LIMIT "+str(limit)

    cur.execute(dml_query)
    
    ### Guardar en un dataframe pandas resultado de consulta

    col_names =  ['news_title', 'news_date_reported','section_name','news_content','news_link']
    my_df  = pd.DataFrame(columns = col_names)

    while True:

        row = cur.fetchone()

        if row == None:
            break
        my_df = my_df.append({'news_title': row[0], 'news_date_reported': row[1], 
                              'section_name': row[2],'news_content': row[3], 'news_link': row[4]}, ignore_index=True)


    con.close()
    
    return (my_df)


# Get lemas from a data.frame for each document
def get_lemma(df):
    
    lista_documentos = []
    for d in range(np.shape(df)[0]):

        doc = nlp(df.loc[d, 'news_content'])

        lda_tokens = [] # lista de tokens limpios

        for token in doc:
            
            # Validate the token (omit stopwords, punctuation, lowercase)
            temp_token = (token.lemma_).lower()
            if not(token.is_stop or token.is_punct or token.is_space):    
                lda_tokens.append(temp_token)

        lista_documentos.append(lda_tokens)
        
    return lista_documentos

In [185]:
# Query for Opinion

# dml_opinion = "SELECT * FROM public.news_content1 WHERE section_name = 'opinion' \
# AND importance_level = 'Nacional' AND news_date_reported > '2018-11-10' LIMIT 250"

dml_opinion = "SELECT DISTINCT news_title,news_date_reported,section_name,news_content, news_link \
FROM public.news_content1 WHERE section_name = 'opinion' AND \
importance_level = 'Nacional' AND news_date_reported > '2018-11-10'"

my_df = get_news_data(dml_opinion,limit=250)  # save into a data frame

In [186]:
## Subsetear un data frame pandas
my_df.loc[1,'news_link']

'https://www.excelsior.com.mx/opinion/raul-contreras-bustamante/70-anos-de-la-declaracion-universal-de-derechos-humanos/1278955'

In [187]:
np.shape(my_df)

(250, 5)

In [6]:
# nlp = spacy.load('es')
# doc = nlp(my_df.loc[3,'news_content'])


# lda_tokens = [] # lista de tokens limpios

# i = 0

# for token in doc:
#     #if token.pos_ =="VERB":
#     if not(token.is_stop or token.is_punct or token.is_space):
#         print(token.text, token.pos_,token.lemma_, token.dep_)
#         i += 1
#         lda_tokens.append(token.lemma_)

# print(i)

In [16]:
# for d in range(np.shape(my_df)[0]):
#     print(d)

In [188]:
nlp = spacy.load('es')  # spanish
nlp.vocab["a"].is_stop = True
nlp.vocab["y"].is_stop = True
nlp.vocab["El"].is_stop = True
nlp.vocab["La"].is_stop = True
nlp.vocab["en"].is_stop = True
nlp.vocab["En"].is_stop = True
nlp.vocab["o"].is_stop = True
nlp.vocab["O"].is_stop = True
# nlp.vocab[""].is_stop = True

In [7]:
lista_documentos = []
numero_noticias = np.shape(my_df)[0]

# Get lemmas skipping punctuation, spaces, and stop words

# for d in range(numero_noticias):

#     doc = nlp(my_df.loc[d, 'news_content'])

#     lda_tokens = [] # lista de tokens limpios

#     i = 0
#     for token in doc:
#         #if token.pos_ =="VERB":
        
#         temp_token = (token.lemma_).lower()
#         if not(token.is_stop or token.is_punct or token.is_space):
#             # print(token.text, token.pos_,token.lemma_, token.dep_)
#             i += 1
#             lda_tokens.append(temp_token)
#     lista_documentos.append(lda_tokens)


In [189]:
lista_documentos=[]
lista_documentos = get_lemma(my_df)

In [194]:
numero_noticias = np.shape(my_df)[0]

In [191]:
# --------------------------------------------------------------------------- #
# Build our dictionary
dictionary = corpora.Dictionary(lista_documentos)

# Check sparsity and other things
dictionary.filter_extremes(no_below = 5, no_above = 0.4)  # tunning dictionary

corpus = [dictionary.doc2bow(text) for text in lista_documentos]

# Save Corpus and Dictionary
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [192]:
# --------------------------------------------------------------------------- #
# LDA

# Note: The used parameters looks very good. Don't touch it!!!

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, 
                                           id2word = dictionary, 
                                           passes=45, eval_every = None,
                                           iterations = 400, alpha = 0.60)
ldamodel.save('model26novadrian.gensim')

topics = ldamodel.print_topics(num_words = 6)
for topic in topics:
    print(topic)

(0, '0.009*"ciento" + 0.008*"banco" + 0.007*"económico" + 0.007*"financiero" + 0.006*"comisionar" + 0.006*"mercar"')
(1, '0.010*"migrante" + 0.009*"guerra" + 0.007*"estados" + 0.006*"unidos" + 0.006*"caravana" + 0.005*"derecho"')
(2, '0.009*"obrador" + 0.007*"partir" + 0.006*"ley" + 0.005*"peña" + 0.005*"consultar" + 0.005*"derecho"')
(3, '0.008*"electoral" + 0.007*"demócrata" + 0.007*"elección" + 0.006*"fiscal" + 0.006*"brasil" + 0.006*"trump"')
(4, '0.012*"seguridad" + 0.006*"público" + 0.005*"policía" + 0.005*"militar" + 0.005*"mexicano" + 0.004*"presentar"')


In [195]:
# --------------------------------------------------------------------------- #
# Get most probably topics for each document in the test set

num_docs = numero_noticias

array_topicos = np.zeros(num_docs)

for id_doc in range(num_docs):
    
    new_doc = lista_documentos[id_doc]  # En realidad esto deberia ser un test set
    new_doc_bow = dictionary.doc2bow(new_doc)  # create a Bag of Words
    most_prob_topic = ldamodel.get_document_topics(new_doc_bow)  # evaluate
    array_topicos[id_doc] = topics[most_prob_topic[0][0]][0]
    
    # print("noticia " + str(i))
    # print(topics[most_prob_topic[0][0]])
    # print(array_topicos[i])
    # print(most_prob_topic)


In [196]:
# Inspect the topics
print(array_topicos)
[[i,my_df.loc[i,'news_link']] for i in range(numero_noticias)]

[1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 2. 0. 1. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 3. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 2. 1. 0. 0. 0. 2. 0. 0. 2. 0.
 0. 0. 0. 0. 2. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 2. 1. 4. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 2. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0. 2. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 4.
 0. 1. 0. 0. 1. 0. 0. 2. 0. 0. 0. 0. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 1. 3. 2. 2. 0. 4. 4. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2. 0. 2. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


[[0, 'https://www.jornada.com.mx/2018/11/15/opinion/029a2pol'],
 [1,
  'https://www.excelsior.com.mx/opinion/raul-contreras-bustamante/70-anos-de-la-declaracion-universal-de-derechos-humanos/1278955'],
 [2,
  'https://www.excelsior.com.mx/opinion/pascal-beltran-del-rio/a-500-anos-de-la-conquista/1278785'],
 [3,
  'https://www.excelsior.com.mx/opinion/adrian-rueda/cuidan-militares-a-negrete/1280225'],
 [4,
  'https://www.excelsior.com.mx/opinion/enrique-aranda/advierte-sandoval-grave-riesgo/1277621'],
 [5, 'https://www.jornada.com.mx/2018/11/17/opinion/026a1mun'],
 [6,
  'https://www.excelsior.com.mx/opinion/enrique-aranda/a-la-suprema-zaldivar/1279694'],
 [7,
  'https://www.excelsior.com.mx/opinion/enrique-aranda/a-la-vista-revuelta-de-rectores/1280713'],
 [8, 'https://www.jornada.com.mx/2018/11/16/opinion/023a2pol'],
 [9,
  'https://www.excelsior.com.mx/opinion/yuriria-sierra/al-tigre-o-a-los-leones/1280424'],
 [10,
  'https://www.excelsior.com.mx/opinion/francisco-garfias/amarga-real

In [29]:
# nlp = spacy.load('es') ### Cargar libreria para NER
# doc1 = nlp(my_df.loc[0,'news_content'])
# doc2 = nlp(my_df.loc[1,'news_content'])
# doc3 = nlp(my_df.loc[2,'news_content'])
# doc4 = nlp(my_df.loc[3,'news_content'])

# for ent in doc4.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [30]:
# print(doc1.similarity(doc2))
# print(doc1.similarity(doc3))
# print(doc1.similarity(doc4))

In [31]:
# nlp = spacy.load('es_core_news_md') ### Cargar libreria para NER
# doc = nlp(my_df.loc[0,'news_content'])

# for ent in doc.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [32]:
# nlp = spacy.load('es_core_news_md') ### Cargar libreria para NER
# doc = nlp(my_df.loc[0,'news_content'])
# np.shape(doc.vector)


In [197]:
#obtener la clasificacion de topicos por cada noticia
#col_names =  ['news_date_reported','section_name','topic','frecuencia']
df_topicos = my_df.copy()
df_topicos["topico"] = np.zeros(np.shape(my_df)[0],dtype=int)

for id_doc in range(0,numero_noticias):
    new_doc = lista_documentos[id_doc]
    new_doc_bow = dictionary.doc2bow(new_doc)
    most_prob_topic=ldamodel.get_document_topics(new_doc_bow)
    df_topicos.loc[id_doc,"topico"] = most_prob_topic[0][0]
    #print(df_topicos.loc[i,"topico"])
    #print(my_df.loc[id_doc,'news_content'])


In [35]:
#df_topicos = my_df.copy()
#df_topicos["topico"] = np.zeros(np.shape(my_df)[0],dtype=int)

In [198]:
#df_topicos.loc[1:100,'topico']
print(df_topicos.iloc[1:5,0:6])

                                          news_title news_date_reported  \
1  70 años de la Declaración Universal de Derecho...         2018-11-17   
2                         A 500 años de la Conquista         2018-11-16   
3          Adrián Rueda - Cuidan militares a Negrete         2018-11-23   
4                  Advierte Sandoval “grave riesgo…”         2018-11-11   

  section_name                                       news_content  \
1      opinion  17 de Noviembre de 2018  \n                   ...   
2      opinion  16 de Noviembre de 2018  \n                   ...   
3      opinion  23 de Noviembre de 2018  \n                   ...   
4      opinion  11 de Noviembre de 2018  \n                   ...   

                                           news_link  topico  
1  https://www.excelsior.com.mx/opinion/raul-cont...       1  
2  https://www.excelsior.com.mx/opinion/pascal-be...       0  
3  https://www.excelsior.com.mx/opinion/adrian-ru...       0  
4  https://www.excelsior.c

In [199]:
#Convertir timestamp to fecha
## Agregar frecuencias por fecha
#pd.to_datetime(df_topicos.news_date_reported[0])
#dfST['timestamp'] = pd.to_datetime(dfST['timestamp'])
#df_topicos.news_date_reported = pd.to_datetime(df)
df_topicos['news_date_reported'] = df_topicos.news_date_reported.dt.date

In [200]:
# subsetear data frames condicionados a un cierto topico
df_topicos[df_topicos.topico ==0][1:10]

,news_title,news_date_reported,section_name,news_content,news_link,topico
3,Adrián Rueda - Cuidan militares a Negrete,2018-11-23,opinion,23 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/adrian-ru...,0
4,Advierte Sandoval “grave riesgo…”,2018-11-11,opinion,11 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/enrique-a...,0
5,A la libertad por la universidad,2018-11-17,opinion,\nEn la imaginaria Ciudad del Sol de Campane...,https://www.jornada.com.mx/2018/11/17/opinion/...,0
6,A la Suprema… ¿Zaldívar?,2018-11-21,opinion,21 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/enrique-a...,0
7,"A la vista, revuelta de rectores…",2018-11-26,opinion,26 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/enrique-a...,0
8,Al fin se anuncia el plan,2018-11-16,opinion,\nEl tan esperado Plan Nacional de Paz y Segur...,https://www.jornada.com.mx/2018/11/16/opinion/...,0
10,“Amarga realidad…”,2018-11-16,opinion,16 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/francisco...,0
11,American curios,2018-11-12,opinion,"\nEn París, mandatarios de varias partes del m...",https://www.jornada.com.mx/2018/11/12/opinion/...,0
13,AMLO es el nuevo Fondo Monetario Internacional,2018-11-15,opinion,15 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/carlos-el...,0


In [201]:
df_topicos[df_topicos.topico ==1][1:10]

,news_title,news_date_reported,section_name,news_content,news_link,topico
1,70 años de la Declaración Universal de Derecho...,2018-11-17,opinion,17 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/raul-cont...,1
9,¿Al tigre o a los leones?,2018-11-24,opinion,24 de Noviembre de 2018 \n ...,https://www.excelsior.com.mx/opinion/yuriria-s...,1
12,American curios,2018-11-26,opinion,"\nNuestro amigo Antonio González, quien fallec...",https://www.jornada.com.mx/2018/11/26/opinion/...,1
19,Aprender a morir,2018-11-20,opinion,\nEn un interesante estudio al que tituló Sen...,https://www.jornada.com.mx/2018/11/20/opinion/...,1
28,Astillero,2018-11-20,opinion,"\nEl juicio a Joaquín Guzmán Loera, en Brook...",https://www.jornada.com.mx/2018/11/20/opinion/...,1
43,Barroco+jarocho... ¡barrocho!,2018-11-18,opinion,"\nSiempre habrá algo de asombroso, y a la vez...",https://www.jornada.com.mx/2018/11/18/opinion/...,1
47,Cambio de régimen,2018-11-17,opinion,\nRecapitulo para avanzar en la conversación. ...,https://www.jornada.com.mx/2018/11/17/opinion/...,1
63,¿Consenso moral?,2018-11-21,opinion,\nSe gesta la constitución moral. Así lo ha r...,https://www.jornada.com.mx/2018/11/21/opinion/...,1
93,Domenico Losurdo: el anticolonial hilo rojo,2018-11-16,opinion,"\nI. Iba a decir que es una tesis, pero con q...",https://www.jornada.com.mx/2018/11/16/opinion/...,1


In [202]:
#df_topicos[df_topicos.topico ==1].group_by["news_date_reported"]
df_topicos["conteo"]=np.ones(np.shape(df_topicos)[0],dtype=int)
df_frecuencia_topicos = df_topicos.groupby(['topico','news_date_reported'])[['conteo']].sum()

df_frecuencia_topicos.reset_index(level=0, inplace=True)
df_frecuencia_topicos.reset_index(level=0, inplace=True)

In [203]:
type(df_frecuencia_topicos)

pandas.core.frame.DataFrame

In [108]:
#df_frecuencia_topicos.columns.values[2]="conteo"

In [204]:
df_frecuencia_topicos

,news_date_reported,topico,conteo
0,2018-11-11,0,5
1,2018-11-12,0,16
2,2018-11-13,0,14
3,2018-11-14,0,9
4,2018-11-15,0,15
5,2018-11-16,0,14
6,2018-11-17,0,15
7,2018-11-18,0,16
8,2018-11-19,0,3
9,2018-11-20,0,10


### Noticias policiacas 

In [205]:
# Query for Policiaca

dml_policiaca = "SELECT DISTINCT news_title, news_date_reported, section_name, \
news_content, news_link FROM public.news_content1 \
WHERE section_name = 'policia' AND news_date_reported > '2018-11-22'"

my_df = get_news_data(dml_policiaca,limit=200)


In [206]:
np.shape(my_df)

(200, 5)

In [207]:
nlp = spacy.load('es')
nlp.vocab["a"].is_stop = True
nlp.vocab["y"].is_stop = True
nlp.vocab["El"].is_stop = True
nlp.vocab["La"].is_stop = True
nlp.vocab["en"].is_stop = True
nlp.vocab["En"].is_stop = True
nlp.vocab["o"].is_stop = True
nlp.vocab["O"].is_stop = True
nlp.vocab["Los"].is_stop = True

lista_documentos = []

lista_documentos = get_lemma(my_df)

# lista_documentos = []

# for d in range(np.shape(my_df)[0]):

#     doc = nlp(my_df.loc[d, 'news_content'])
    
#     lda_tokens = [] # lista de tokens limpios
    
#     for token in doc:
#         temp_token = (token.lemma_).lower()
#         # Validate the token (omit stopwords, punctuation)
#         if not(token.is_stop or token.is_punct or token.is_space):    
#             lda_tokens.append(temp_token)
            
#     lista_documentos.append(lda_tokens)


In [208]:
# Build bag of words

token_list = list(lista_documentos)
my_dict = corpora.Dictionary(token_list)
my_dict.filter_extremes(no_below = 6, no_above = 0.35) # clean dictionary

my_corpus = [my_dict.doc2bow(text) for text in token_list]  # transform into a Bag of Words

#corpus = models.TfidfModel(corpus)
print('Number of unique tokens from my_dict: %d' % len(my_dict))
print('Number of documents in my_corpus: %d' % len(my_corpus))

Number of unique tokens from my_dict: 614
Number of documents in my_corpus: 200


In [209]:
# --------------------------------------------------------------------------- #

# LDA Parameters

NUM_TOPICS = 5

passes = 45  # like epochs
eval_every = None  # NA
iters = 400 # iterations
eta = 0.60 # NA

# Build LDA Model
ldamodel2 = gensim.models.ldamodel.LdaModel(my_corpus, num_topics = NUM_TOPICS, 
                                            id2word = my_dict, passes = passes,
                                            eval_every = eval_every,
                                            iterations = iters,
                                            eta = eta)
ldamodel2.save('model26novAARVpolicia.gensim')

In [125]:
# Check BOW for some document

#id2word = my_dict
# [[(id2word[id], freq) for id, freq in cp] for cp in my_corpus[:2]]


In [210]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel2, my_corpus, id2word)


/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [211]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.096956  0.015427       1        1  42.170273
2      0.096658 -0.001745       2        1  19.935474
3     -0.022252  0.074833       3        1  17.751637
1     -0.022052 -0.109183       4        1  13.474378
0      0.044602  0.020670       5        1   6.668240, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
158   Default   40.000000          menor   40.000000  30.0000  30.0000
161   Default   47.000000              n   47.000000  29.0000  29.0000
132   Default   61.000000  investigación   61.000000  28.0000  28.0000
42    Default   70.000000           zona   70.000000  27.0000  27.0000
338   Default   21.000000      incendiar   21.000000  26.0000  26.0000
30    Default   29.000000      policíaco   29.000000  25.0000  25.0000
170   Default   55.000000         ubicar   55.000000  24.0000  24.0000
13    Default   36.000000          civil   36.000000  23.0000  23.0000
290   Default  101.000000         hombre  101.000000  22.0000  22.0000
147   Default   41.000000          casar   41.000000  21.0000  21.0000
328   Default   29.000000     protección   29.000000  20.0000  20.0000
60    Default   24.000000           hijo   24.000000  19.0000  19.0000
221   Default   30.000000      operativo   30.000000  18.0000  18.0000
73    Default   49.000000          salir   49.000000  17.0000  17.0000
28    Default   52.000000          mujer   52.000000  16.0000  16.0000
26    Default   32.000000    ministerial   32.000000  15.0000  15.0000
97    Default   75.000000      municipal   75.000000  14.0000  14.0000
194   Default   31.000000         delito   31.000000  13.0000  13.0000
229   Default   28.000000          robar   28.000000  12.0000  12.0000
487   Default   16.000000         dinero   16.000000  11.0000  11.0000
64    Default   19.000000          madre   19.000000  10.0000  10.0000
168   Default   26.000000      solicitar   26.000000   9.0000   9.0000
5     Default   86.000000          armar   86.000000   8.0000   8.0000
214   Default   20.000000            mil   20.000000   7.0000   7.0000
154   Default   36.000000       fiscalía   36.000000   6.0000   6.0000
239   Default   17.000000          venta   17.000000   5.0000   5.0000
51    Default   54.000000           edad   54.000000   4.0000   4.0000
286   Default   29.000000    disposición   29.000000   3.0000   3.0000
164   Default   17.000000           peso   17.000000   2.0000   2.0000
86    Default   39.000000         ciudad   39.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
304    Topic5    2.933923        agentes   10.636012   1.4199  -5.5718
384    Topic5    2.469603          señor    9.596563   1.3505  -5.7440
169    Topic5    3.155557       teléfono   12.268332   1.3500  -5.4989
596    Topic5    3.648738         privar   14.406637   1.3345  -5.3537
191    Topic5    2.378136       criminal    9.614789   1.3108  -5.7818
149    Topic5    1.783601      contactar    7.270908   1.3026  -6.0695
168    Topic5    6.445080      solicitar   26.358820   1.2993  -4.7848
413    Topic5    3.154825        pistola   12.907963   1.2989  -5.4992
457    Topic5    5.079936        centrar   21.743076   1.2538  -5.0228
158    Topic5    8.960909          menor   40.825069   1.1914  -4.4552
64     Topic5    4.526203          madre   19.878162   1.2281  -5.1382
132    Topic5   10.778909  investigación   61.432251   0.9675  -4.2705
147    Topic5    7.650521          casar   41.297531   1.0218  -4.6133
170    Topic5    9.346325         ubicar   55.365116   0.9288  -4.4131
73     Topic5    7.760167          salir   49.577824   0.8533  -4.5991
161    Topic5    7.240864              n   47.573765   0.8253  -4.6684
164    Topic5    3.888830           peso   17.091293   1.2274  -5.2900
34     Topic5    5.220042    

In [212]:
num_words = 7
topics2 = ldamodel2.print_topics(num_words = num_words)
for topic in topics2:
    print(topic)

(0, '0.014*"investigación" + 0.012*"ubicar" + 0.012*"menor" + 0.010*"salir" + 0.010*"hombre" + 0.010*"casar" + 0.009*"n"')
(1, '0.015*"zona" + 0.012*"incendiar" + 0.012*"policíaco" + 0.012*"civil" + 0.012*"armar" + 0.010*"protección" + 0.010*"ministerial"')
(2, '0.012*"seguridad" + 0.011*"n" + 0.010*"municipal" + 0.010*"estatal" + 0.009*"agente" + 0.009*"colonia" + 0.009*"disposición"')
(3, '0.013*"encontrar" + 0.012*"mujer" + 0.011*"menor" + 0.010*"edad" + 0.010*"hijo" + 0.010*"hecho" + 0.009*"lesionar"')
(4, '0.014*"hombre" + 0.012*"vehículo" + 0.010*"reportar" + 0.010*"lesionar" + 0.010*"unidad" + 0.009*"conductor" + 0.009*"trasladar"')


In [213]:
new_doc = lista_documentos[15]
new_doc
new_doc_bow = dictionary.doc2bow(new_doc)
new_doc_bow
ldamodel2.get_document_topics(new_doc_bow)

IndexError: index 667 is out of bounds for axis 1 with size 614

In [155]:
#obtener la clasificacion de topicos por cada noticia
#col_names =  ['news_date_reported','section_name','topic','frecuencia']
df_topicos_pol = my_df.copy()
df_topicos_pol["topico"] = np.zeros(np.shape(my_df)[0],dtype=int)
most_prob_topic=None

for id_doc in range(0,np.shape(my_df)[0]):
    new_doc = lista_documentos[id_doc]
    new_doc_bow = dictionary.doc2bow(new_doc)
    #print(np.shape(new_doc_bow))
    most_prob_topic=ldamodel2.get_document_topics(new_doc_bow)
    #df_topicos.loc[id_doc,"topico"] = most_prob_topic[0][0]
    #print(df_topicos.loc[i,"topico"])
    #print(my_df.loc[id_doc,'news_content'])

IndexError: index 755 is out of bounds for axis 1 with size 614

In [151]:
np.shape(my_df)[0]
most_prob_topic
len(lista_documentos)

200

In [129]:
#df_topicos_pol = my_df.copy()
#df_topicos_pol["topico"] = np.zeros(np.shape(my_df)[0],dtype=int)

In [130]:
#df_topicos.loc[1:100,'topico']
print(df_topicos_pol.iloc[1:5,0:6])

                                          news_title news_date_reported  \
1  Velocidad terminó con la vida de Rebeca Alejandra         2018-11-23   
2             Derrapa vehículo y alcanza a camioneta         2018-11-26   
3  Conflicto vial termina en ataque a balazos en ...         2018-11-24   
4    Buscan a menor desaparecida en San Juan del Río         2018-11-26   

  section_name                                       news_content  \
1      policia  Amigos y familiares despidieron Rebeca Alejand...   
2      policia  Córdoba, Ver.- Un choque por alcance entre dos...   
3      policia  Un supuesto conflicto vial, terminó en disparo...   
4      policia  Una madre solicita el apoyo de la ciudadanía p...   

                                           news_link  topico  
1  https://www.elheraldodechihuahua.com.mx/polici...       0  
2  https://www.elsoldecordoba.com.mx/policiaca/de...       0  
3  https://www.elsoldeleon.com.mx/policiaca/confl...       0  
4  https://www.elsoldesanj

In [131]:
#Convertir timestamp to fecha
## Agregar frecuencias por fecha
#pd.to_datetime(df_topicos.news_date_reported[0])
#dfST['timestamp'] = pd.to_datetime(dfST['timestamp'])
#df_topicos.news_date_reported = pd.to_datetime(df)
df_topicos_pol['news_date_reported'] = df_topicos_pol.news_date_reported.dt.date

In [134]:
# subsetear data frames condicionados a un cierto topico
df_topicos_pol[df_topicos_pol.topico ==0][1:10]

,news_title,news_date_reported,section_name,news_content,news_link,topico
1,Velocidad terminó con la vida de Rebeca Alejandra,2018-11-23,policia,Amigos y familiares despidieron Rebeca Alejand...,https://www.elheraldodechihuahua.com.mx/polici...,0
2,Derrapa vehículo y alcanza a camioneta,2018-11-26,policia,"Córdoba, Ver.- Un choque por alcance entre dos...",https://www.elsoldecordoba.com.mx/policiaca/de...,0
3,Conflicto vial termina en ataque a balazos en ...,2018-11-24,policia,"Un supuesto conflicto vial, terminó en disparo...",https://www.elsoldeleon.com.mx/policiaca/confl...,0
4,Buscan a menor desaparecida en San Juan del Río,2018-11-26,policia,Una madre solicita el apoyo de la ciudadanía p...,https://www.elsoldesanjuandelrio.com.mx/polici...,0
5,Detienen a 63 personas durante operativos en Z...,2018-11-24,policia,"Zumpango, México.- Un total de 63 personas, a ...",https://www.elsoldetoluca.com.mx/policiaca/det...,0
6,Fuga de gasolina causa daño cerca de invernaderos,2018-11-25,policia,Severos daños ambientales causó la fuga de hid...,https://www.elsoldehidalgo.com.mx/policiaca/fu...,0
7,Se incendia camioneta en la colonia Miramar,2018-11-24,policia,Tremendo susto se llevó una conductora al ince...,https://www.elsoldetampico.com.mx/policiaca/se...,0
8,Muere joven atropellado por urbano,2018-11-23,policia,Un hombre de 29 años murió en la Cruz Verde de...,https://www.eloccidental.com.mx/policiaca/muer...,0
9,"Atacó a un taxista, estuvieron a punto de linc...",2018-11-23,policia,"Por los presuntos delitos de lesiones, daños y...",https://www.elsoldesanluis.com.mx/policiaca/at...,0


In [139]:
df_topicos_pol[df_topicos_pol.topico ==4]

,news_title,news_date_reported,section_name,news_content,news_link,topico


In [114]:
#df_topicos[df_topicos.topico ==1].group_by["news_date_reported"]
df_topicos["conteo"]=np.ones(np.shape(df_topicos)[0],dtype=int)
df_frecuencia_topicos = df_topicos.groupby(['topico','news_date_reported'])[['conteo']].sum()

df_frecuencia_topicos.reset_index(level=0, inplace=True)
df_frecuencia_topicos.reset_index(level=0, inplace=True)

In [112]:
type(df_frecuencia_topicos)

pandas.core.frame.DataFrame

In [108]:
#df_frecuencia_topicos.columns.values[2]="conteo"

In [115]:
df_frecuencia_topicos

,news_date_reported,topico,conteo
0,2018-11-11,0,3
1,2018-11-12,0,12
2,2018-11-13,0,8
3,2018-11-14,0,6
4,2018-11-15,0,10
5,2018-11-16,0,14
6,2018-11-17,0,17
7,2018-11-18,0,17
8,2018-11-19,0,1
9,2018-11-20,0,10
